### Overview of this notebook

* overview of `sktime` as a framework - estimators modules, library, data
* overview of learning tasks in `sktime` - API perspective
* searching the library for estimators, tag system
* estimator level dependency management
* creating your own estimator, to `sktime`, or for third party use (closed or open) - short primer

In [16]:
import warnings
warnings.filterwarnings("ignore")

# 5. `sktime` in a nutshell - engineering perspective

**A) `sktime` is a modular framework for multiple learning tasks**

Example: forecasting (predict future of ts), classification (predict label of ts)

**B) estimators/algorithms are of a scientific type = which task do they solve?**

Example: ARIMA is a forecaster; knn with time series distance is classifier

**C) all estimators of a certain scitype have the same module interface**

Example: all forecasters classes have `fit` / `predict` with same contract

**D) `sktime` is a library which allows browsing of integrated estimators**

Example: search for all forecasters that are natively multivariate

**E) `sktime` is a mini-package manager for estimators and their dependencies**

Example: `ARIMA` class requires `pmdarima`, but `sktime` itself does not

**F) `sktime` is extensible, write your own 3rd party plugins (open or closed)**

Example: forecaster in 3rd party codebase, plug & plays to `sktime` and test framework

**G) `sktime` is highly composable - any compatible estimators of any type!**

Example: pipeline 3rd party feature extractor and onboard detrender with forecaster

## 5.1 Showcase with code vignettes

The above, with code. We revisit in more detail later.

### **A) `sktime` is a modular framework for multiple learning tasks**

Vignettes for forecasting and classification:
(we'll go into data types etc later)

In [17]:
from sktime.datasets import load_airline
from sktime.forecasting.naive import NaiveForecaster
import numpy as np

# step 1: data specification
y = load_airline()
# y is a pd.Series at monthly frequency

# step 2: specifying forecasting horizon
fh = np.arange(1, 37)
# this specifies a prediction 3 years ahead

# step 3: specifying the forecasting algorithm
forecaster = NaiveForecaster(strategy="last", sp=12)
# forecaster is now a forecaster object of type NaiveForecaster

# step 4: fitting the forecaster
forecaster.fit(y, fh=fh)
# forecaster changes state to "fitted"

# step 5: querying predictions
y_pred = forecaster.predict()
# y_pred is the forecasted time series, a pd.Series

In [18]:
from sktime.datasets import load_osuleaf
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels.compose_tab_to_panel import AggrDist
from sktime.dists_kernels import ScipyDist

# step 1 - specify training data
X_train, y_train = load_osuleaf(split="train", return_type="numpy3D")
# X_train is 3D numpy array holding multiple instances of time series
# y_train is 1D numpy array with training labels for these instances

# step 2 - specify data to predict labels for
X_new, _ = load_osuleaf(split="test", return_type="numpy3D")
X_new = X_new[:2]
# X_new is a 3D numpy array with the instances to label

# step 3 - specify the classifier
mean_eucl_dist = AggrDist(ScipyDist())
clf = KNeighborsTimeSeriesClassifier(n_neighbors=3, distance=mean_eucl_dist)
# clf is a classifier object of type KNeighborsTimeSeriesClassifier
# it consists of other sktime objects, mean_eucl_dist is a distance object

# step 4 - fitting the classifier
clf.fit(X_train, y_train)
# clf changes state to "fitted"

# step 5 - predict labels on new data
y_pred = clf.predict(X_new)
# y_pred is the predicted labels, an 1D numpy array

### **B) estimators/algorithms are of a scientific type = which task do they solve?**

`NaiveForecaster` is a forecaster; `KNeighborsTimeSeriesClassifier` is a classifier

In [19]:
from sktime.forecasting.naive import NaiveForecaster
from sktime.registry import scitype

scitype(NaiveForecaster)

'forecaster'

In [20]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.registry import scitype

scitype(KNeighborsTimeSeriesClassifier)

'classifier'

**C) all estimators of a certain scitype have the same module interface**

C1 - the `NaiveForecaster` can be switched out for any forecaster in the base vignette

Only step 3 - specification changes!

In [21]:
from sktime.datasets import load_airline
from sktime.forecasting.arima import ARIMA
import numpy as np

# step 1: data specification
y = load_airline()

# step 2: specifying forecasting horizon
fh = np.arange(1, 37)

# step 3: specifying the forecasting algorithm
# forecaster = NaiveForecaster(strategy="last", sp=12)
forecaster = ARIMA()

# step 4: fitting the forecaster
forecaster.fit(y)

# step 5: querying predictions
y_pred = forecaster.predict(fh)

C2 - `fit` and `predict` are the same for both!

`fit(self, y, X=None, fh=None)`

In [22]:
?ARIMA.fit

Signature: ARIMA.fit(self, y, X=None, fh=None)
Docstring:
Fit forecaster to training data.

State change:
    Changes state to "fitted".

Writes to self:
    Sets self._is_fitted flag to True.
    Writes self._y and self._X with `y` and `X`, respectively.
    Sets self.cutoff and self._cutoff to last index seen in `y`.
    Sets fitted model attributes ending in "_".
    Stores fh to self.fh if fh is passed.

Parameters
----------
y : time series in sktime compatible data container format
        Time series to which to fit the forecaster.
    y can be in one of the following formats:
    Series scitype: pd.Series, pd.DataFrame, or np.ndarray (1D or 2D)
        for vanilla forecasting, one time series
    Panel scitype: pd.DataFrame with 2-level row MultiIndex,
        3D np.ndarray, list of Series pd.DataFrame, or nested pd.DataFrame
        for global or panel forecasting
    Hierarchical scitype: pd.DataFrame with 3 or more level row MultiIndex
        for hierarchical forecasting
  

In [23]:
?NaiveForecaster.fit

Signature: NaiveForecaster.fit(self, y, X=None, fh=None)
Docstring:
Fit forecaster to training data.

State change:
    Changes state to "fitted".

Writes to self:
    Sets self._is_fitted flag to True.
    Writes self._y and self._X with `y` and `X`, respectively.
    Sets self.cutoff and self._cutoff to last index seen in `y`.
    Sets fitted model attributes ending in "_".
    Stores fh to self.fh if fh is passed.

Parameters
----------
y : time series in sktime compatible data container format
        Time series to which to fit the forecaster.
    y can be in one of the following formats:
    Series scitype: pd.Series, pd.DataFrame, or np.ndarray (1D or 2D)
        for vanilla forecasting, one time series
    Panel scitype: pd.DataFrame with 2-level row MultiIndex,
        3D np.ndarray, list of Series pd.DataFrame, or nested pd.DataFrame
        for global or panel forecasting
    Hierarchical scitype: pd.DataFrame with 3 or more level row MultiIndex
        for hierarchical fore

for classifiers, signature is different: `fit(X, y)`

but the same for all classifiers!

In [24]:
KNeighborsTimeSeriesClassifier.fit

<function sktime.classification.base.BaseClassifier.fit(self, X, y)>

### **D) `sktime` is a library which allows browsing of integrated estimators**

Example: search for all forecasters that can make probabilistic predictions

In [25]:
from sktime.registry import all_estimators

all_estimators("forecaster", filter_tags={"capability:pred_int": True}, as_dataframe=True)

,name,object
0,ARIMA,<class 'sktime.forecasting.arima.ARIMA'>
1,AutoARIMA,<class 'sktime.forecasting.arima.AutoARIMA'>
2,AutoETS,<class 'sktime.forecasting.ets.AutoETS'>
3,BATS,<class 'sktime.forecasting.bats.BATS'>
4,BaggingForecaster,<class 'sktime.forecasting.compose._bagging.Ba...
5,ColumnEnsembleForecaster,<class 'sktime.forecasting.compose._column_ens...
6,ConformalIntervals,<class 'sktime.forecasting.conformal.Conformal...
7,DynamicFactor,<class 'sktime.forecasting.dynamic_factor.Dyna...
8,ForecastX,<class 'sktime.forecasting.compose._pipeline.F...
9,ForecastingGridSearchCV,<class 'sktime.forecasting.model_selection._tu...


all objects in `sktime` are tagged with metadata:

In [26]:
ARIMA().get_tags()

{'scitype:y': 'univariate',
 'ignores-exogeneous-X': False,
 'capability:insample': True,
 'capability:pred_int': True,
 'capability:pred_int:insample': True,
 'handles-missing-data': True,
 'y_inner_mtype': 'pd.Series',
 'X_inner_mtype': 'pd.DataFrame',
 'requires-fh-in-fit': False,
 'X-y-must-have-same-index': True,
 'enforce_index_type': None,
 'fit_is_empty': False,
 'python_version': None,
 'python_dependencies': 'pmdarima'}

list all tags that apply to forecasters:

In [27]:
from sktime.registry import all_tags

all_tags("forecaster", as_dataframe=True)

,name,scitype,type,description
0,X-y-must-have-same-index,"[forecaster, regressor]",bool,do X/y in fit/update and X/fh in predict have ...
1,X_inner_mtype,"[forecaster, transformer, transformer-pairwise...","(list, [pd.Series, pd.DataFrame, np.array, nes...",which machine type(s) is the internal _fit/_pr...
2,capability:insample,forecaster,bool,can the forecaster make in-sample predictions?
3,capability:pred_int,forecaster,bool,does the forecaster implement predict_interval...
4,capability:pred_int:insample,forecaster,bool,can the forecaster make in-sample predictions ...
5,capability:pred_var,forecaster,bool,does the forecaster implement predict_variance?
6,enforce_index_type,"[forecaster, regressor]",type,"passed to input checks, input conversion index..."
7,ignores-exogeneous-X,forecaster,bool,does forecaster ignore exogeneous data (X)?
8,remember_data,"[forecaster, transformer]",bool,whether estimator remembers all data seen as s...
9,requires-fh-in-fit,forecaster,bool,does forecaster require fh passed already in f...


### **E) `sktime` is a mini-package manager for estimators and their dependencies**

Example: `ARIMA` class requires `pmdarima`, but `sktime` itself does not

In [28]:
from sktime.forecasting.arima import ARIMA

ARIMA.get_class_tag("python_dependencies")
# this requires the pmdarima package
# the result is a PEP 440 compatible requirement string

'pmdarima'

by default, dependencies are checked at instantiation:

In [29]:
# from sktime.forecasting.fbprophet import Prophet

# Prophet()

# this would result in:

this would result in exception:

```
ModuleNotFoundError: Prophet requires package 'prophet' to be present in the python environment,
but 'prophet' was not found. 'prophet' is a soft dependency and not included in the base
sktime installation. Please run: `pip install prophet` to install the prophet package.
To install all soft dependencies, run: `pip install sktime[all_extras]`
```

### **F) `sktime` is extensible, write your own 3rd party plugins (open or closed)**

Note: To check your own estimator you need to have [`pytest`](https://github.com/pytest-dev/pytest) installed.

Example: forecaster in 3rd party codebase, plug & plays to `sktime` and test framework

snippet from forecaster extension template (in `extension_templates` dir):
```
How to use this implementation template to implement a new estimator:
- make a copy of the template in a suitable location, give it a descriptive name.
- work through all the "todo" comments below
- fill in code for mandatory methods, and optionally for optional methods
- do not write to reserved variables: is_fitted, _is_fitted, _X, _y, cutoff, _fh,
    _cutoff, _converter_store_y, forecasters_, _tags, _tags_dynamic, _is_vectorized
- you can add more private methods, but do not override BaseEstimator's private methods
    an easy way to be safe is to prefix your methods with "_custom"
- change docstrings for functions and the file
- ensure interface compatibility by sktime.utils.estimator_checks.check_estimator
- once complete: use as a local library, or contribute to sktime via PR
- more details:
  https://www.sktime.net/en/stable/developer_guide/add_estimators.html
```

![](./img/implementing_estimators.png)

In [30]:
from sktime.transformations.series.boxcox import BoxCoxTransformer
from sktime.utils.estimator_checks import check_estimator

res = check_estimator(BoxCoxTransformer)

In [31]:
res

{'test_clone[BoxCoxTransformer]': 'PASSED',
 'test_constructor[BoxCoxTransformer]': 'PASSED',
 'test_create_test_instance[BoxCoxTransformer]': 'PASSED',
 'test_create_test_instances_and_names[BoxCoxTransformer]': 'PASSED',
 'test_estimator_tags[BoxCoxTransformer]': 'PASSED',
 'test_get_params[BoxCoxTransformer]': 'PASSED',
 'test_get_test_params[BoxCoxTransformer]': 'PASSED',
 'test_has_common_interface[BoxCoxTransformer]': 'PASSED',
 'test_inheritance[BoxCoxTransformer]': 'PASSED',
 'test_no_between_test_case_side_effects[BoxCoxTransformer-TransformerFitTransformSeriesUnivariate-0]': 'PASSED',
 'test_no_between_test_case_side_effects[BoxCoxTransformer-TransformerFitTransformSeriesUnivariate-1]': 'PASSED',
 'test_no_between_test_case_side_effects[BoxCoxTransformer-TransformerFitTransformSeriesMultivariate-0]': 'PASSED',
 'test_no_between_test_case_side_effects[BoxCoxTransformer-TransformerFitTransformSeriesMultivariate-1]': 'PASSED',
 'test_no_between_test_case_side_effects[BoxCoxTrans

**G) `sktime` is highly composable - any compatible estimators of any type!**

Example: pipeline 3rd party feature extractor and onboard detrender with forecaster

In [32]:
from sklearn.linear_model import LinearRegression

from sktime.forecasting.arima import ARIMA
from sktime.forecasting.trend import TrendForecaster
from sktime.transformations.series.boxcox import BoxCoxTransformer
from sktime.transformations.series.detrend import Detrender

trend_reg = LinearRegression()
detrender = Detrender(TrendForecaster(trend_reg))

pipe = BoxCoxTransformer() * detrender * ARIMA()

# we first box-cox-transform - from statsmodels
# then we detrend - sktime native
# detrending is done using a regression forecaster - sktime native
# the regression in the trend forecaster is linear regressor - from sklearn
# then we apply ARIMA - from pmdarima

# when forecasting, the detrending and box cox transform are inverted, in this order

# this is how it looks like - highly modular!
# any step could be switched out and can be tuned with grid search etc
pipe

TransformedTargetForecaster(steps=[BoxCoxTransformer(),
                                   Detrender(forecaster=TrendForecaster(regressor=LinearRegression())),
                                   ARIMA()])

In [33]:
# tunable parameters
pipe.get_params()

{'steps': [BoxCoxTransformer(),
  Detrender(forecaster=TrendForecaster(regressor=LinearRegression())),
  ARIMA()],
 'BoxCoxTransformer': BoxCoxTransformer(),
 'Detrender': Detrender(forecaster=TrendForecaster(regressor=LinearRegression())),
 'ARIMA': ARIMA(),
 'BoxCoxTransformer__bounds': None,
 'BoxCoxTransformer__method': 'mle',
 'BoxCoxTransformer__sp': None,
 'Detrender__forecaster': TrendForecaster(regressor=LinearRegression()),
 'Detrender__model': 'additive',
 'Detrender__forecaster__regressor': LinearRegression(),
 'Detrender__forecaster__regressor__copy_X': True,
 'Detrender__forecaster__regressor__fit_intercept': True,
 'Detrender__forecaster__regressor__n_jobs': None,
 'Detrender__forecaster__regressor__positive': False,
 'ARIMA__concentrate_scale': False,
 'ARIMA__enforce_invertibility': True,
 'ARIMA__enforce_stationarity': True,
 'ARIMA__hamilton_representation': False,
 'ARIMA__maxiter': 50,
 'ARIMA__measurement_error': False,
 'ARIMA__method': 'lbfgs',
 'ARIMA__mle_regr

## 5.2 Learning tasks in sktime

Step 1 - what is your learning task?

sktime estimator type support:

| Task | Status | Links |
|---|---|---|
| **Forecasting** | stable | [Tutorial](https://www.sktime.net/en/latest/examples/01_forecasting.html) · [API Reference](https://www.sktime.net/en/latest/api_reference/forecasting.html) · [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/forecasting.py)  |
| **Time Series Classification** | stable | [Tutorial](https://github.com/sktime/sktime/blob/main/examples/02_classification.ipynb) · [API Reference](https://www.sktime.net/en/latest/api_reference/classification.html) · [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/classification.py) |
| **Time Series Regression** | stable | [API Reference](https://www.sktime.net/en/latest/api_reference/regression.html) |
| **Transformations** | stable | [Tutorial](https://github.com/sktime/sktime/blob/main/examples/03_transformers.ipynb) · [API Reference](https://www.sktime.net/en/latest/api_reference/transformations.html) · [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/transformer.py)  |
| **Parameter fitting** | maturing | [API Reference](https://www.sktime.net/en/latest/api_reference/param_est.html) · [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/transformer.py)  |
| **Time Series Clustering** | maturing | [API Reference](https://www.sktime.net/en/latest/api_reference/clustering.html) ·  [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/clustering.py) |
| **Time Series Distances/Kernels** | maturing | [Tutorial](https://github.com/sktime/sktime/blob/main/examples/03_transformers.ipynb) · [API Reference](https://www.sktime.net/en/latest/api_reference/dists_kernels.html) · [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/dist_kern_panel.py) |
| **Annotation** | experimental | [Extension Template](https://github.com/sktime/sktime/blob/main/extension_templates/annotation.py) |
| **Distributions and simulation** | experimental |  |

rough overview of time series related learning tasks

![](./img/ts-tasks.jpg)

first some basic terminology on time series (required for the above)

### 5.2.1 learning task guide - primer

questions to ask:

1. what is my data? one time series, panel of time series?
2. what do I want to do with it: predict? transform? annotate?
3. what data comes out of the model when it is applied?
4. how do I train? on what data? (or do I not train?)

starting with explaining common tasks:

* forecasting
* simple panel data tasks - classification, regression, clustering
* panel forecasting

#### **forecasting** (vanilla)

![](./img/tasks-forecasting.png)

e.g., **given this**:

In [34]:
from sktime.datasets import load_macroeconomic

y = load_macroeconomic()
y

,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
Period,,,,,,,,,,,,
1959Q1,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0.00,0.00
1959Q2,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
1959Q3,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
1959Q4,2785.204,1753.7,299.356,484.052,1931.3,29.370,140.0,4.33,5.6,179.386,0.27,4.06
1960Q1,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
...,...,...,...,...,...,...,...,...,...,...,...,...
2008Q3,13324.600,9267.7,1990.693,991.551,9838.3,216.889,1474.7,1.17,6.0,305.270,-3.16,4.33
2008Q4,13141.920,9195.3,1857.661,1007.273,9920.4,212.174,1576.5,0.12,6.9,305.952,-8.79,8.91
2009Q1,12925.410,9209.2,1558.494,996.287,9926.4,212.671,1592.8,0.22,8.1,306.547,0.94,-0.71


**algorithm, please fill in rows for 2009Q4, 2010Q1, ...**

if want only some columns (variables) for 2009Q4:

* columns to forecast are called "endogenous"
* other columns, if used by algorithm, are called "exogenous"
* some exogenous columns may be known for future (e.g., interest rates), some may be unknown

#### **time series classification** and regression

![](./img/tasks-tsc.png)

e.g., **given this**:

In [35]:
from sktime.datasets import load_basic_motions

X, _ = load_basic_motions(split="TEST", return_type="pd-multiindex")
X

dim_0      dim_1      dim_2      dim_3     dim_4     dim_5
   timepoints                                                                
0  0           -0.740653   0.756509  -0.275809  -0.423476  0.013317  0.013317
   1           -0.740653   0.756509  -0.275809  -0.423476  0.013317  0.013317
   2           10.208449  -9.216970 -12.378901 -14.699153  4.578337 -5.055081
   3            2.867009  -5.977115  -6.540994   5.561122  2.178639 -1.624657
   4           -0.194301  -3.711996  -0.795126  -1.728529 -1.054696 -0.276991
...                  ...        ...        ...        ...       ...       ...
39 95          28.459024 -16.633770   3.631869   8.978229 -3.611533 -1.491489
   96          10.260094   0.102775   1.269261  -1.645964 -3.377157  1.283746
   97           4.316471  -3.574319   2.063831  -1.717875 -1.843054  0.484734
   98           0.704446  -4.920444   2.851857  -2.982977 -0.809665 -0.721774
   99          -2.074749  -6.892377   4.848379  -1.350330 -1.203844 -1.776470

[4000 rows x 6 columns]

**algorithm, please assign a label to instance 0, a label to instance 1, ..., to instance 39**

label is a category (e.g., unifoliate leaf, bifoliate leaf, ...) -> time series classification

label is a number (e.g., 1, 3.14, 42) -> time series regression

**you can train on some instances with known label**:

In [36]:
from sktime.datasets import load_basic_motions

X_train, y_train = load_basic_motions(split="TRAIN", return_type="pd-multiindex")
X_train, y_train

(                  dim_0     dim_1     dim_2     dim_3     dim_4     dim_5
    timepoints                                                            
 0  0           0.079106  0.394032  0.551444  0.351565  0.023970  0.633883
    1           0.079106  0.394032  0.551444  0.351565  0.023970  0.633883
    2          -0.903497 -3.666397 -0.282844 -0.095881 -0.319605  0.972131
    3           1.116125 -0.656101  0.333118  1.624657 -0.569962  1.209171
    4           1.638200  1.405135  0.393875  1.187864 -0.271664  1.739182
 ...                 ...       ...       ...       ...       ...       ...
 39 95          1.239144 -6.142442  0.028264 -2.309144  1.472845 -0.998765
    96          0.261434  0.205915 -0.224944 -0.524684  0.769715  0.157139
    97          2.490353 -0.878765 -0.597296  0.111862 -0.117188 -0.050604
    98          4.122120  0.911620 -0.465409  0.535338  0.197090  0.442120
    99          3.169270  0.826934 -0.362036 -0.298298  0.250357  0.428803
 
 [4000 rows x 6 columns

#### **time series clustering**

data as above, but there is no labelling or training data

algorithm should do one or more of of:

* put the series into a small number of buckets (prototype 1, prototype 2, ...)
* compute a similarity hierarchy between the instances
* produce a parsimonious clustering model summary (algorithm dependent)

#### **panel forecasting**

= for a panel of time series, forecast all (or only some) of the instances

e.g., **given this**:

In [37]:
from sktime.utils._testing.hierarchical import _make_hierarchical

y = _make_hierarchical(hierarchy_levels=(4,), min_timepoints=4, max_timepoints=5)
y

c0
h0   time                
h0_0 2000-01-02  3.289729
     2000-01-03  3.711591
     2000-01-04  2.946715
     2000-01-05  3.275697
h0_1 2000-01-02  1.936790
     2000-01-03  1.000000
     2000-01-04  1.629057
     2000-01-05  3.777519
h0_2 2000-01-02  3.283345
     2000-01-03  4.318675
     2000-01-04  1.036824
     2000-01-05  1.722950
h0_3 2000-01-02  1.900095
     2000-01-03  1.950562
     2000-01-04  5.960328
     2000-01-05  2.594919

**algorithm, please forecast 2000-01-06, 2000-01-07 for instances h0_0, h0_1, ...**

optional: full training instances are available that are not being forecast (sometimes called"global" forecasting)

IMPORTANT: not the same as time series regression! Not the same as tabular regression!

Forecasts are numbers, but they are indexed by time - regression label is not time indexed.

#### further and rarer learning tasks

for disambiguation - experimental or no support in `sktime` (contribute!)

* prediction is also time series
    * concurrent on same time axis: supervised annotation, nowcasting
    * future, same time axis: global or supervised forecasting
    * unrelated axis: functional regression
* prediction is summary or transform of time series
    * directional forecasting
    * annotation forecast
* detecting anomalies, changepoints
    * temporal anomaly detection, changepoint detection
    * latent space modelling, unsupervised annotation

most above tasks can be:

* panel/hierarchical or not
* supervised or not, training data varying across index
* batch or on-line
* combined with output modality: point prediction, probabilistic, event

`sktime` roadmap to cover "task space" in order of commonality

contributions appreciated!

### 5.2.1 Time Series Forecasting

Basic deployment vignette for forecasting:

1. load/setup training data, `y` as `pd.DataFrame`
2. specify the forecasting horizon, i.e., what to forecast
3. specify the forecaster using `sklearn`-like syntax
4. fit forecaster to training data, `fit(y, fh)`
5. compute forecasts, `predict()`

In [38]:
# step 1 - prepare macroeconomic data
from sktime.datasets import load_macroeconomic

y = load_macroeconomic().iloc[:-1]  # we leave out the last row for later

In [39]:
y

,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
Period,,,,,,,,,,,,
1959Q1,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0.00,0.00
1959Q2,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
1959Q3,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
1959Q4,2785.204,1753.7,299.356,484.052,1931.3,29.370,140.0,4.33,5.6,179.386,0.27,4.06
1960Q1,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
...,...,...,...,...,...,...,...,...,...,...,...,...
2008Q2,13415.266,9351.0,2026.518,961.280,10059.0,218.610,1409.3,1.74,5.4,304.483,8.53,-6.79
2008Q3,13324.600,9267.7,1990.693,991.551,9838.3,216.889,1474.7,1.17,6.0,305.270,-3.16,4.33
2008Q4,13141.920,9195.3,1857.661,1007.273,9920.4,212.174,1576.5,0.12,6.9,305.952,-8.79,8.91


In [40]:
# step 2 - specify the forecasting horizon
fh = [1, 2, 3, 4]  # four quarters = 1 full year ahead

In [41]:
# step 3 - specify the forecasting algorithm
from sktime.forecasting.naive import NaiveForecaster

# this predicts the last value seen
forecaster = NaiveForecaster(strategy="last")

we could specify any `sktime` forecaster here - the rest remains the same!

In [42]:
# step 4 - fit the forecaster to the historical data
forecaster.fit(y, fh=fh)

NaiveForecaster()

In [43]:
# the forecaster is now fitted
forecaster.is_fitted

True

In [44]:
# and we can inspect fitted parameters if we like
forecaster.get_fitted_params()

{'forecasters': 1                          cpi               infl                 m1  \
 0                                                                      
 forecasters  NaiveForecaster()  NaiveForecaster()  NaiveForecaster()   
 
 1                          pop           realcons            realdpi  \
 0                                                                      
 forecasters  NaiveForecaster()  NaiveForecaster()  NaiveForecaster()   
 
 1                      realgdp           realgovt            realint  \
 0                                                                      
 forecasters  NaiveForecaster()  NaiveForecaster()  NaiveForecaster()   
 
 1                      realinv           tbilrate              unemp  
 0                                                                     
 forecasters  NaiveForecaster()  NaiveForecaster()  NaiveForecaster()  ,
 "forecasters.loc['forecasters','cpi']": NaiveForecaster(),
 "forecasters.loc['forecasters','cpi']__sp": 

In [45]:
# step 5 - compute forecasts
y_pred = forecaster.predict()

In [46]:
y_pred

,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
Period,,,,,,,,,,,,
2009Q3,214.469,3.37,1653.6,307.226,9189.0,10077.5,12901.504,1023.528,-3.19,1456.678,0.18,9.2
2009Q4,214.469,3.37,1653.6,307.226,9189.0,10077.5,12901.504,1023.528,-3.19,1456.678,0.18,9.2
2010Q1,214.469,3.37,1653.6,307.226,9189.0,10077.5,12901.504,1023.528,-3.19,1456.678,0.18,9.2
2010Q2,214.469,3.37,1653.6,307.226,9189.0,10077.5,12901.504,1023.528,-3.19,1456.678,0.18,9.2


all together in one cell:

In [47]:
from sktime.datasets import load_macroeconomic

from sktime.forecasting.naive import NaiveForecaster

# step 1 - prepare macroeconomic data
y = load_macroeconomic()

# step 2 - specify the forecasting horizon
fh = [1, 2, 3, 4]  # four quarters = 1 full year ahead

# step 3 - specify the forecasting algorithm
forecaster = NaiveForecaster(strategy="last")

# step 4 - fit the forecaster to the historical data
forecaster.fit(y, fh=fh)

# step 5 - compute forecasts
y_pred = forecaster.predict()

sktime allows online/stream mode with forecaster update:

suppose tine now passes and we get new data in 2009Q3

In [48]:
y_new = load_macroeconomic().iloc[-1:]  # the row we left out, let's pretend it's new

In [49]:
y_new

,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
Period,,,,,,,,,,,,
2009Q3,12990.341,9256.0,1486.398,1044.088,10040.6,216.385,1673.9,0.12,9.6,308.013,3.56,-3.44


we can update the forecaster with it:

In [50]:
forecaster.update(y_new)

NaiveForecaster()

and we can get new forecasts, again for one year ahead (the forecaster remembers the `fh`):

In [51]:
forecaster.predict()

,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
Period,,,,,,,,,,,,
2009Q4,216.385,3.56,1673.9,308.013,9256.0,10040.6,12990.341,1044.088,-3.44,1486.398,0.12,9.6
2010Q1,216.385,3.56,1673.9,308.013,9256.0,10040.6,12990.341,1044.088,-3.44,1486.398,0.12,9.6
2010Q2,216.385,3.56,1673.9,308.013,9256.0,10040.6,12990.341,1044.088,-3.44,1486.398,0.12,9.6
2010Q3,216.385,3.56,1673.9,308.013,9256.0,10040.6,12990.341,1044.088,-3.44,1486.398,0.12,9.6


### 5.2.2 Panel forecasting

panel forecasting does not have a separate estimator type. All forecasters can do this.

Passing panel data is enough for this:

In [52]:
from sktime.forecasting.naive import NaiveForecaster
from sktime.utils._testing.hierarchical import _make_hierarchical

# step 1 - prepare some panel data
y = _make_hierarchical(hierarchy_levels=(4,), min_timepoints=4, max_timepoints=4)

# step 2 - specify the forecasting horizon
fh = [1, 2, 3, 4]  # four quarters = 1 full year ahead

# step 3 - specify the forecasting algorithm
forecaster = NaiveForecaster(strategy="last")

# step 4 - fit the forecaster to the historical data
forecaster.fit(y, fh=fh)

# step 5 - compute forecasts
y_pred = forecaster.predict()

In [53]:
y.head()

c0
h0   time                
h0_0 2000-01-01  3.177179
     2000-01-02  4.063708
     2000-01-03  1.760755
     2000-01-04  3.370000
h0_1 2000-01-01  3.862318

In [54]:
y_pred.head()

c0
h0   time               
h0_0 2000-01-05  3.37000
     2000-01-06  3.37000
     2000-01-07  3.37000
     2000-01-08  3.37000
h0_1 2000-01-05  3.74513

### 5.2.3 Time Series Classification

Basic deployment vignette for TSC:

1. load/setup training data, `X` in a `Panel` format, `y` as 1D `np.ndarray`
2. load/setup new data for prediction (can be done after 2 too)
3. specify the classifier using `sklearn`-like syntax
4. fit classifier to training data, `fit(X, y)`
5. predict labels on new data, `predict(X_new)`

In [55]:
# steps 1, 2 - prepare osuleaf dataset (train and new)
from sktime.datasets import load_osuleaf

X_train, y_train = load_osuleaf(split="train", return_type="numpy3D")
X_new, _ = load_osuleaf(split="test", return_type="numpy3D")
X_new = X_new[:2]  # smaller dataset for faster notebook runtime

In [56]:
# this is in numpy3D format, but could also be pd-multiindex or other
X_train.shape

(200, 1, 427)

In [57]:
# y is a 1D np.ndarray of labels - same length as number of instances in X_train
y_train.shape

(200,)

In [58]:
# step 3 - specify the classifier
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier

# example 1 - 3-NN with simple dynamic time warping distance (requires numba)
clf = KNeighborsTimeSeriesClassifier(n_neighbors=3)

# example 2:
# 3-nearest neighbour classifier with mean (over time points) pairwise Euclidean distance
# (requires scipy)
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels.compose_tab_to_panel import AggrDist
from sktime.dists_kernels import ScipyDist

mean_eucl_dist = AggrDist(ScipyDist())
clf = KNeighborsTimeSeriesClassifier(n_neighbors=3, distance=mean_eucl_dist)

we could specify any `sktime` classifier here - the rest remains the same!

In [59]:
# all classifiers is scikit-learn / scikit-base compatible!
# nested parameter interface via get_params, set_params
clf.get_params()

{'algorithm': 'brute',
 'distance': AggrDist(transformer=ScipyDist()),
 'distance_mtype': None,
 'distance_params': None,
 'leaf_size': 30,
 'n_jobs': None,
 'n_neighbors': 3,
 'pass_train_distances': False,
 'weights': 'uniform',
 'distance__aggfunc': None,
 'distance__aggfunc_is_symm': False,
 'distance__transformer': ScipyDist(),
 'distance__transformer__colalign': 'intersect',
 'distance__transformer__metric': 'euclidean',
 'distance__transformer__metric_kwargs': None,
 'distance__transformer__p': 2,
 'distance__transformer__var_weights': None}

In [60]:
# step 4 - fit/train the classifier
clf.fit(X_train, y_train)

KNeighborsTimeSeriesClassifier(distance=AggrDist(transformer=ScipyDist()),
                               n_neighbors=3)

In [61]:
# the classifier is now fitted
clf.is_fitted

True

In [62]:
# and we can inspect fitted parameters if we like
clf.get_fitted_params()

{'classes': array(['1', '2', '3', '4', '5', '6'], dtype='<U1'),
 'fit_time': 3,
 'knn_estimator': KNeighborsClassifier(algorithm='brute', metric='precomputed', n_neighbors=3),
 'n_classes': 6,
 'knn_estimator__classes': array(['1', '2', '3', '4', '5', '6'], dtype='<U1'),
 'knn_estimator__effective_metric': 'precomputed',
 'knn_estimator__effective_metric_params': {},
 'knn_estimator__n_features_in': 200,
 'knn_estimator__n_samples_fit': 200,
 'knn_estimator__outputs_2d': False}

In [63]:
# step 5 - predict labels on new data
y_pred = clf.predict(X_new)

In [64]:
# y_pred is an 1D np.ndarray, similar to sklearn classification output
y_pred

array(['1', '3'], dtype='<U1')

all together in one cell:

In [65]:
# steps 1, 2 - prepare osuleaf dataset (train and new)
from sktime.datasets import load_osuleaf

X_train, y_train = load_osuleaf(split="train", return_type="numpy3D")
X_new, _ = load_osuleaf(split="test", return_type="numpy3D")
X_new = X_new[:2]  # smaller dataset for faster notebook runtime

# step 3 - specify the classifier
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels.compose_tab_to_panel import AggrDist
from sktime.dists_kernels import ScipyDist

mean_eucl_dist = AggrDist(ScipyDist())
clf = KNeighborsTimeSeriesClassifier(n_neighbors=3, distance=mean_eucl_dist)

# step 4 - fit/train the classifier
clf.fit(X_train, y_train)

# step 5 - predict labels on new data
y_pred = clf.predict(X_new)

evaluation same as in sklearn - use `accuracy` etc

### 5.2.4 Time Series Regression

TSR vignettes are exactly the same as TSC, except that:

* `y` in `fit` input and `predict` output should be float 1D `np.ndarray`, not categorical
* other algorithms are commonly used and/or performant

In [66]:
# steps 1, 2 - prepare dataset (train and new)
from sktime.datasets import load_covid_3month

X_train, y_train = load_covid_3month(split="train")
y_train = y_train.astype("float")
X_new, _ = load_covid_3month(split="test")
X_new = X_new.loc[:2]  # smaller dataset for faster notebook runtime

# step 3 - specify the regressor
from sktime.regression.distance_based import KNeighborsTimeSeriesRegressor

clf = KNeighborsTimeSeriesRegressor(n_neighbors=3, distance=mean_eucl_dist)

# step 4 - fit/train the regressor
clf.fit(X_train, y_train)

# step 5 - predict labels on new data
y_pred = clf.predict(X_new)

In [67]:
y_pred  # not too interesting but float

array([0., 0., 0.])

### 5.2.5 Time Series Clustering

TS clustering is similar - 1st step is also `fit`, but unsupervised

i.e., no labels `y`, and next step is inspecting clusters

In [68]:
from sktime.clustering.dbscan import TimeSeriesDBSCAN

# step 1 - prepare dataset (train and new)
X, _ = load_osuleaf(split="train", return_type="numpy3D")
X = X[:10]

# step 2 - specify the clusterer
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels.compose_tab_to_panel import AggrDist
from sktime.dists_kernels import ScipyDist

mean_eucl_dist = AggrDist(ScipyDist())
clst = TimeSeriesDBSCAN(distance=mean_eucl_dist)

# step 3 - fit the clusterer to the data
clst.fit(X)

# step 4 - inspect the clustering
clst.get_fitted_params()

{'core_sample_indices': array([], dtype=int64),
 'dbscan': DBSCAN(metric='precomputed'),
 'fit_time': 102,
 'labels': array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]),
 'dbscan__components': array([], shape=(0, 10), dtype=float64),
 'dbscan__core_sample_indices': array([], dtype=int64),
 'dbscan__labels': array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]),
 'dbscan__n_features_in': 10}

## 5.3 Searching for estimators, estimator tags

Estimators in `sktime` are tagged.

User facing tags indicate things the estimator can or cannot do, these usually
start with "capability" (but not all, for historical reasons).

Examples:

forecasters

* `"capability:pred_int"` - can make probabilistic predictions (`predict_interval` etc)
* `"ignores-exogeneous-X"` - can make use of exogeneous data or not
* `"handles-missing-data"` - dealing with missing values

classifiers

* `"capability:missing_values"` - dealing with missing values
* `"capability:multivariate"` - daeling with multivariate input
* `"capability:unequal_length"` - dealing with time series panels where the individual time series have unequal length and/or unequal index

all tags for an estimator scitype (e.g., classifier, regressor) can be inspected by `sktime.registry.all_tags`:

In [69]:
from sktime.registry import all_tags

all_tags("classifier", as_dataframe=True)

,name,scitype,type,description
0,capability:contractable,classifier,bool,"contract time setting, does the estimator supp..."
1,capability:missing_values,"[classifier, early_classifier, param_est, regr...",bool,"can the classifier handle missing data (NA, np..."
2,capability:multithreading,"[classifier, early_classifier]",bool,can the classifier set n_jobs to use multiple ...
3,capability:multivariate,"[classifier, early_classifier, param_est, regr...",bool,can the classifier classify time series with 2...
4,capability:predict_proba,classifier,bool,does the classifier implement a non-default pr...
5,capability:train_estimate,classifier,bool,can the classifier estimate its performance on...
6,capability:unequal_length,"[classifier, early_classifier, regressor, tran...",bool,can the estimator handle unequal length time s...
7,classifier_type,classifier,"(list, [dictionary, distance, feature, hybrid,...",which type the classifier falls under in the t...


valid estimator types are listed in the `all_tags` docstring, or `sktime.registry.BASE_CLASS_REGISTER`

In [70]:
from sktime.registry import BASE_CLASS_REGISTER

# get only fist table column, the list of types
list(zip(*BASE_CLASS_REGISTER))[0]

('object',
 'estimator',
 'aligner',
 'classifier',
 'clusterer',
 'early_classifier',
 'forecaster',
 'metric',
 'network',
 'param_est',
 'regressor',
 'series-annotator',
 'splitter',
 'transformer',
 'transformer-pairwise',
 'transformer-pairwise-panel',
 'distribution')

to find all estimators of a certain type, use `sktime.registry.all_estimators`

In [71]:
# list all forecasters in sktime
from sktime.registry import all_estimators

all_estimators("forecaster", as_dataframe=True)

,name,object
0,ARDL,<class 'sktime.forecasting.ardl.ARDL'>
1,ARIMA,<class 'sktime.forecasting.arima.ARIMA'>
2,AutoARIMA,<class 'sktime.forecasting.arima.AutoARIMA'>
3,AutoETS,<class 'sktime.forecasting.ets.AutoETS'>
4,AutoEnsembleForecaster,<class 'sktime.forecasting.compose._ensemble.A...
5,BATS,<class 'sktime.forecasting.bats.BATS'>
6,BaggingForecaster,<class 'sktime.forecasting.compose._bagging.Ba...
7,ColumnEnsembleForecaster,<class 'sktime.forecasting.compose._column_ens...
8,ConformalIntervals,<class 'sktime.forecasting.conformal.Conformal...
9,Croston,<class 'sktime.forecasting.croston.Croston'>


for listing all estimators of a certain type with a certain capability,
use the `filter_tags` argument of `all_estimators`:

In [72]:
# list all forecasters in sktime
# that can make probabilistic forecasts, e.g., prediction interval forecasts
from sktime.registry import all_estimators

all_estimators("forecaster", as_dataframe=True, filter_tags={"capability:pred_int": True})

,name,object
0,ARIMA,<class 'sktime.forecasting.arima.ARIMA'>
1,AutoARIMA,<class 'sktime.forecasting.arima.AutoARIMA'>
2,AutoETS,<class 'sktime.forecasting.ets.AutoETS'>
3,BATS,<class 'sktime.forecasting.bats.BATS'>
4,BaggingForecaster,<class 'sktime.forecasting.compose._bagging.Ba...
5,ColumnEnsembleForecaster,<class 'sktime.forecasting.compose._column_ens...
6,ConformalIntervals,<class 'sktime.forecasting.conformal.Conformal...
7,DynamicFactor,<class 'sktime.forecasting.dynamic_factor.Dyna...
8,ForecastX,<class 'sktime.forecasting.compose._pipeline.F...
9,ForecastingGridSearchCV,<class 'sktime.forecasting.model_selection._tu...


## 5.4 Custom estimators - extension guide

![](./img/implementing_estimators.png)

`sktime` is meant to be easily extensible, for direct contribution to `sktime` as well as for local/private extension with custom methods.

To extend `sktime` with a new local or contributed estimator, a good workflow to follow is:

0. find the right extension template for the type of estimator you want to add - e.g., classifier, regressor, clusterer, etc. The extension templates are located in the [`extension_templates`](https://github.com/sktime/sktime/blob/main/extension_templates) directory
1. read through the extension template - this is a `python` file with `todo` blocks that mark the places in which changes need to be added.
2. optionally, if you are planning any major surgeries to the interface: look at the base class - note that "ordinary" extension (e.g., new algorithm) should be easily doable without this.
3. copy the extension template to a local folder in your own repository (local/private extension), or to a suitable location in your clone of the `sktime` or affiliated repository (if contributed extension), inside `sktime.[name_of_task]`; rename the file and update the file docstring appropriately.
4. address the "todo" parts. Usually, this means: changing the name of the class, setting the tag values, specifying hyper-parameters, filling in `__init__`, `_fit`, `_predict` and/or other methods (for details see the extension template). You can add private methods as long as they do not override the default public interface. For more details, see the extension template.
5. to test your estimator manually: import your estimator and run it in the basic vignettes above.
6. to test your estimator automatically: call `sktime.tests.test_all_estimators.check_estimator` on your estimator. You can call this on a class or object instance. Ensure you have specified test parameters in the `get_test_params` method, according to the extension template.

In case of direct contribution to `sktime` or one of its affiliated packages, additionally:
* add yourself as an author to the code, and to the `CODEOWNERS` for the new estimator file(s).
* create a pull request that contains only the new estimators (and their inheritance tree, if it's not just one class), as well as the automated tests as described above.
* in the pull request, describe the estimator and optimally provide a publication or other technical reference for the strategy it implements.
* before making the pull request, ensure that you have all necessary permissions to contribute the code to a permissive license (BSD-3) open source project.

---

### Credits: notebook 5 - sktime engineering perspective

notebook creation: fkiraly

General credit to `sklearn` and `sktime` contributors